# *1. Data*

In [ ]:
import pandas as pd 
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
df_hero = pd.read_csv("hero-network.csv")

In [ ]:
df_hero.head(5)

In [ ]:
df_nodes = pd.read_csv("nodes.csv")

In [ ]:
df_nodes.head(5)

In [ ]:
df_edges = pd.read_csv("edges.csv")

In [ ]:
df_edges.head(5)

## Preprocessing

In [ ]:
df_hero["hero1"] = df_hero["hero1"].apply(lambda x: x.strip('/'))
df_hero["hero2"] = df_hero["hero2"].apply(lambda x: x.strip('/'))
df_hero["hero1"] = df_hero["hero1"].apply(lambda x: x.strip())
df_hero["hero2"] = df_hero["hero2"].apply(lambda x: x.strip())

In [ ]:
df_edges["hero"] = df_edges["hero"].apply(lambda x: x.strip('/'))
df_edges["hero"] = df_edges["hero"].apply(lambda x: x.strip())

In [ ]:
df_nodes["node"] = df_nodes["node"].apply(lambda x: x.strip('/'))
df_nodes["node"] = df_nodes["node"].apply(lambda x: x.strip())

In [ ]:
df_hero.tail(5)

In [ ]:
df_edges["hero"] = df_edges["hero"].apply(lambda x: x[:20])

In [ ]:
df_edges.loc[75723]

In [ ]:
df_hero_noduplicate = df_hero[df_hero.hero1 != df_hero.hero2].copy(deep=True)

In [ ]:
df_hero_noduplicate.shape

In [ ]:
df_hero.shape

In [ ]:
df_edges.to_pickle("edges_clear.pkl")
df_hero_noduplicate.to_pickle("hero_clear.pkl")
df_nodes.to_pickle('nodes_clear.pkl')

In [ ]:
df_hero_noduplicate = pd.read_pickle("hero_clear.pkl")
df_edges = pd.read_pickle("edges_clear.pkl")
df_nodes = pd.read_pickle("nodes_clear.pkl")

## **Graphs setup**

In [ ]:
Graph_1 = nx.MultiGraph()

In [ ]:
list(set((df_hero_noduplicate.hero1).unique()).union(set((df_hero_noduplicate.hero2).unique())))

In [ ]:
Graph_1.add_nodes_from(list(set((df_hero_noduplicate.hero1).unique()).union(set((df_hero_noduplicate.hero2).unique()))))

In [ ]:
Graph_1.add_edges_from(list(zip(list(df_hero_noduplicate.hero1),list(df_hero_noduplicate.hero2))))

In [ ]:
nx.info(Graph_1)

In [ ]:
# create weighted graph from M
Graph_1reduce = nx.Graph()
for u,v,w in Graph_1.edges:
    if Graph_1reduce.has_edge(u,v):
        Graph_1reduce[u][v]['weight'] += 1
    else:
        Graph_1reduce.add_edge(u, v, weight=1)

In [ ]:
for u,v in Graph_1reduce.edges:
    Graph_1reduce[u][v]['weight']=1/Graph_1reduce[u][v]['weight']    

In [ ]:
Graph_1reduce.edges()[('PRINCESS ZANDA', 'LITTLE, ABNER')]['weight']

### Graph_2

In [ ]:
Graph_2 = nx.Graph()

In [ ]:
Graph_2.add_nodes_from(list((df_nodes.node).unique()))

In [ ]:
Graph_2.add_edges_from(list(zip(list(df_edges.hero),list(df_edges.comic))))

In [ ]:
nx.info(Graph_2)

### Functionality 3 - Shortest ordered Route
Input:

* The graph data
* A sequence of superheroes h = [h_2, ..., h_n-1]
* Initial node h_1 and an end node h_n
* N: denoting the top N heroes that their data should be considered

Output:
* The shortest walk of comics that you need to read to get from hero_1 to hero_n 

Considerations: For this functionality, you need to implement an algorithm that returns the shortest walk that goes from node h_j to h_n, which\
 visits in order the nodes in h. The choice of h_j and h_n can be made randomly (or if it improves the performance of the algorithm, you can also\
 define it in any other way)

Important Notes:

* This algorithm should be run only on the second graph.
* The algorithm needs to handle the case that the graph is not connected. Thus, only some of the nodes in h are reachable from h_1. In such \
a scenario, it is enough to let the program give in the output the string "There is no such path".
* Since we are dealing with walks, you can pass on the same node h_i more than once, but you have to preserve order. E.g., if you start from\
 Spiderman to reach deadpool, and your path requires you to visit iron-man and colossus, you can go back to any comics any time you \
 want, assuming that the order in which you visit the heroes is still the same.

In [ ]:
def Top_N_heroes(N,df_edges):
    degree_nodes = df_edges.groupby('hero').size().reset_index()
    degree_nodes.rename(columns={0: "size"},inplace=True)
    first_N_heroes = list((degree_nodes.sort_values(by="size",ascending=False).head(N).hero))
    return(first_N_heroes)

In [ ]:
def Shortest_Path_Comics(graph_data,seq_heroes,init_node,end_node,N):
    Shortest_OR = [init_node]
    first_N_heroes1 = Top_N_heroes(N,graph_data)
    New_dataset = df_edges[df_edges['hero'].isin(first_N_heroes1)]
    Graph_3 = nx.Graph()
    Graph_3.add_edges_from(list(zip(list(New_dataset.hero),list(New_dataset.comic))))
    Graph_3_copy = Graph_3.copy()
    seq_heroes.append(end_node)
    for i in range(1,len(seq_heroes)):
        if((Graph_3_copy.has_node(seq_heroes[i-1])) & (Graph_3_copy.has_node(seq_heroes[i]))):
            Shortest_OR += nx.shortest_path(Graph_3_copy,seq_heroes[i-1],seq_heroes[i])[1:]
            #[Graph_3_copy.remove_nodes_from(Shortest_OR[:-1])
        else:
            print("There is no such path.")
            break
    return(Shortest_OR[1:-1])


In [ ]:
first_N_heroes1 = Top_N_heroes(N,graph_data)
New_dataset = df_edges[df_edges['hero'].isin(first_N_heroes1)]
Graph_3 = nx.Graph()
Graph_3.add_edges_from(list(zip(list(New_dataset.hero),list(New_dataset.comic))))
Graph_3_copy = Graph_3.copy()

In [ ]:
graph_data = input("Enter the desidered graph:(In this case only 'df_edges' is allowed) ")
if graph_data=='df_edges':
    graph_data = df_edges
else:
    print("Re-run the cell, input not correct")
    sys.exit()
N = int(input("Enter the top N heroes that we should considered: "))
if(N<0 or N>Graph_2.number_of_nodes()):
    N=Graph_2.number_of_nodes()
first_N_heroes = Top_N_heroes(N,graph_data)
init_nodo = input("Enter the initial hero node: ")
if(init_nodo not in first_N_heroes):
    print("Re-run the cell, input not correct")
    sys.exit()
end_node = input("Enter the hero end node: ")
if(end_node not in first_N_heroes):
    print("Re-run the cell, input not correct")
    sys.exit()
seq_heroes = []
n = int(input("Enter the list size \n"))
for i in range(0, n):
    print("Enter hero at index", i, )
    hero= input()
    seq_heroes.append(hero)
print("User list is ", seq_heroes)
for elem in seq_heroes:
    if elem not in first_N_heroes:
        print("Re-run the cell, input not correct")
        sys.exit()


In [32]:
Shortest_OR = Shortest_Path_Comics(graph_data,seq_heroes,init_nodo,end_node,N)
print(f'This is the connecting component we have found that start with {init_nodo} and ends at {end_node}:\n'
            ,Shortest_OR)
print("If we want to visualize only a walk of comics then: \n",[item for item in Shortest_OR if item not in first_N_heroes])

There is no such path.
This is the connecting component we have found that start with SPIDER-MAN/PETER PAR and ends at WOLVERINE/LOGAN:
 []
If we want to visualize only a walk of comics then: 
 []
